In [3]:

import pandas as pd

In [4]:
df = pd.read_parquet(r"C:\Users\Admin\Desktop\New folder\post_data.parquet")

In [5]:
df.head(5)

,screen_name,hashtags,bookmark_count,created_at,favorite_count,full_text,is_quote_status,quote_count,reply_count,retweet_count,...,is_reply,views,url,time_diff,average_time_diff,time_diff_seconds,average_time_diff_seconds,time_bot,views/like,views_score
0,0xASMedia,HOODIESEASON,1,2023-11-10 19:32:06,151,#HOODIESEASON\n\nFollowing trends always kind ...,False,1,24,209,...,False,2814,https://twitter.com/0xASMedia/status/172306098...,NaN,40029.5,NaN,40029.5,NaN,18.635762,0
1,0xASMedia,Giveaway,1,2023-11-11 13:40:51,312,ASM x Unchained Elephants🔥\n\nTake part in a #...,False,1,85,308,...,False,5079,https://twitter.com/0xASMedia/status/172333498...,65325.0,40029.5,65325.0,40029.5,0.0,16.278846,0
2,0xASMedia,XRP,1,2023-11-11 13:40:51,312,ASM x Unchained Elephants🔥\n\nTake part in a #...,False,1,85,308,...,False,5079,https://twitter.com/0xASMedia/status/172333498...,0.0,40029.5,0.0,40029.5,0.0,16.278846,0
3,0xASMedia,auction,1,2023-11-11 13:40:51,312,ASM x Unchained Elephants🔥\n\nTake part in a #...,False,1,85,308,...,False,5079,https://twitter.com/0xASMedia/status/172333498...,0.0,40029.5,0.0,40029.5,0.0,16.278846,0
4,0xASMedia,XRP,1,2023-11-13 14:15:03,289,The time is approaching quickly! #XRP #ASM\n\n...,False,1,230,331,...,False,4615,https://twitter.com/0xASMedia/status/172406836...,174852.0,40029.5,174852.0,40029.5,0.0,15.968858,0


In [6]:
df.shape

(14811, 24)

In [7]:
df.isnull().sum()

screen_name                     0
hashtags                        0
bookmark_count                  0
created_at                      0
favorite_count                  0
full_text                       0
is_quote_status                 0
quote_count                     0
reply_count                     0
retweet_count                   0
user_id_str                     0
id                              0
is_retweet                      0
is_quoted                       0
is_reply                        0
views                           0
url                             0
time_diff                    1241
average_time_diff             652
time_diff_seconds            1241
average_time_diff_seconds     652
time_bot                     1241
views/like                      0
views_score                     0
dtype: int64

In [8]:
df = df[['screen_name','hashtags','full_text']]

In [9]:
df.head(5)

,screen_name,hashtags,full_text
0,0xASMedia,HOODIESEASON,#HOODIESEASON\n\nFollowing trends always kind ...
1,0xASMedia,Giveaway,ASM x Unchained Elephants🔥\n\nTake part in a #...
2,0xASMedia,XRP,ASM x Unchained Elephants🔥\n\nTake part in a #...
3,0xASMedia,auction,ASM x Unchained Elephants🔥\n\nTake part in a #...
4,0xASMedia,XRP,The time is approaching quickly! #XRP #ASM\n\n...


In [10]:
df['full_text'][0]

'#HOODIESEASON\n\nFollowing trends always kind of pissed me off\n\nLet it come from the heart💔\n\nPortion of the profits from all the clothing drops distributed back to Lost Souls holders, I think they call that "utility"\n\nStill reading? Like, follow &amp; retweet to enter into a 25 XRP… https://t.co/F1id1DVpRe'

In [11]:
df['full_text'] = df['full_text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)


In [12]:
df['full_text'][0]

'#hoodieseason  following trends always kind of pissed me off  let it come from the heart💔  portion of the profits from all the clothing drops distributed back to lost souls holders, i think they call that "utility"  still reading? like, follow &amp; retweet to enter into a 25 xrp… https://t.co/f1id1dvpre'

In [13]:
import nltk
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
df['full_text'] = df['full_text'].apply(lambda x: tokenization(x))

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['full_text'])
similarity = cosine_similarity(matrix)

In [45]:
similarity[0]

array([1.        , 0.06864192, 0.06864192, ..., 0.00291609, 0.00291609,
       0.05372947])

In [17]:
def recommendation(screen_name):
    idx = df[df['screen_name'] == screen_name].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    recommends = []
    for m_id in distances[1:20]:
        if df.iloc[m_id[0]].full_text in recommends:
          continue
        recommends.append(df.iloc[m_id[0]].full_text)
        if len(recommends) == 4:
          break
        
    return recommends

In [18]:
recommendation('0xASMedia')

['the time is approach quickli ! # xrp # asm # giveaway follow , retweet , & amp ; tag the homi for a chanc to win 15 xrp ! there are onli 108 lost soul left to be mint ! ⏳ we are close in on the final opportun to be elig for our genesi phygit + 3d metavers collectibles… http : //t.co/oe7xlleqz7',
 'win a share of $ 1,000 # bitcoin to enter : - like 👍 - retweet ♻️ - follow @ cryptosavingexp & amp ; @ martiniguyyt winner drawn on friday',
 '# shibarmi would you like to see a portion of the proce from the @ tangem shiba wallet burn ? let your voic be heard in the poll below ! 🔥',
 '4/ alway dyor — and more to come on thi soon 👀']

In [19]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))